In [1]:
import sqlite3
import pandas as pd

# Adjust this path if necessary
db_path = "db/immune_cells.db"

conn = sqlite3.connect(db_path)

print("Connected to:", db_path)

# -----------------------
# Load raw database tables
# -----------------------
df_projects = pd.read_sql_query("SELECT * FROM projects ORDER BY project_id;", conn)
df_subjects = pd.read_sql_query("SELECT * FROM subjects ORDER BY subject_id;", conn)
df_samples = pd.read_sql_query("SELECT * FROM samples ORDER BY sample_id;", conn)

print("\n=== Projects ===")
display(df_projects)

print("\n=== Subjects ===")
display(df_subjects)

print("\n=== Samples ===")
display(df_samples)

# -----------------------
# Integrity checks
# -----------------------

# 1. Subjects that do not match any project
orphan_subjects = pd.read_sql_query("""
    SELECT subject_id, subject, project_id
    FROM subjects
    WHERE project_id NOT IN (SELECT project_id FROM projects);
""", conn)

print("\n=== Subjects referencing missing projects ===")
display(orphan_subjects)

# 2. Samples whose subject_id is invalid
orphan_samples = pd.read_sql_query("""
    SELECT sample_id, sample, subject_id
    FROM samples
    WHERE subject_id NOT IN (SELECT subject_id FROM subjects);
""", conn)

print("\n=== Samples referencing missing subjects ===")
display(orphan_samples)

# 3. Count samples per project (via join)
samples_per_project = pd.read_sql_query("""
    SELECT p.project_name, COUNT(s.sample_id) AS n_samples
    FROM projects p
    LEFT JOIN subjects subj ON subj.project_id = p.project_id
    LEFT JOIN samples s ON s.subject_id = subj.subject_id
    GROUP BY p.project_id
    ORDER BY p.project_name;
""", conn)

print("\n=== Sample count per project ===")
display(samples_per_project)

conn.close()


Connected to: db/immune_cells.db

=== Projects ===


,project_id,project_name
0,1,prj1
1,2,prj2
2,3,prj3



=== Subjects ===


,subject_id,subject,project_id,treatment,response,sample_type,condition,sex
0,1,sbj000,1,miraclib,no,PBMC,melanoma,M
1,2,sbj001,1,miraclib,yes,PBMC,carcinoma,M
2,3,sbj002,1,miraclib,no,PBMC,melanoma,M
3,4,sbj003,1,miraclib,no,PBMC,melanoma,M
4,5,sbj004,1,miraclib,yes,PBMC,carcinoma,M
...,...,...,...,...,...,...,...,...
3495,3496,sbj3495,3,miraclib,no,PBMC,carcinoma,M
3496,3497,sbj3496,3,phauximab,no,PBMC,carcinoma,M
3497,3498,sbj3497,3,miraclib,yes,PBMC,melanoma,M
3498,3499,sbj3498,3,phauximab,no,PBMC,melanoma,M



=== Samples ===


...

,subject_id,subject,project_id



=== Samples referencing missing subjects ===


,sample_id,sample,subject_id



=== Sample count per project ===


,project_name,n_samples
0,prj1,4500
1,prj2,3000
2,prj3,3000
